# `p2oc` CLI Demo

Open a double-funded channel without the aid of an intermediary being present.

In [1]:
%load_ext autoreload
%autoreload 2

## Fund Ali and Bob

In [2]:
import sys
sys.path.append('/home/jovyan/lnrpc')
sys.path.append('/notebooks/p2oc/p2oc')

import os
import time
import bitcoin
from btc_rpc import Proxy, Config
from lnd_rpc import LndRpc, lnmsg

In [3]:
%%sh
sudo cp /ali-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/ali.macaroon
sudo chmod +r /tmp/ali.macaroon

sudo cp /bob-lnd/data/chain/bitcoin/regtest/admin.macaroon /tmp/bob.macaroon
sudo chmod +r /tmp/bob.macaroon

In [4]:
ali = LndRpc(host='ali-lnd:10009', cert_path='/ali-lnd/tls.cert', macaroon_path='/tmp/ali.macaroon')
bob = LndRpc(host='bob-lnd:10009', cert_path='/bob-lnd/tls.cert', macaroon_path='/tmp/bob.macaroon')

bitcoin.SelectParams('regtest')

brpc = Proxy(config=Config(
    rpcuser=os.environ['BTCD_RPCUSER'],
    rpcpassword=os.environ['BTCD_RPCPASS'],
    rpcconnect=os.environ['BTCD_RPCHOST'],
    rpcport=os.environ['BTCD_RPCPORT']
))

In [5]:
ali_address = ali.lnd.NewAddress(lnmsg.NewAddressRequest(type=lnmsg.AddressType.WITNESS_PUBKEY_HASH))
bob_address = bob.lnd.NewAddress(lnmsg.NewAddressRequest(type=lnmsg.AddressType.WITNESS_PUBKEY_HASH))

try:
    brpc.loadwallet('miner')
except:
    brpc.createwallet('miner')
    
miner_address = brpc.getnewaddress("coinbase")
brpc.unloadwallet('miner')

{'warning': ''}

In [6]:
_ = list(brpc.generatetoaddress(400, miner_address))
_ = list(brpc.generatetoaddress(10, ali_address.address))
_ = list(brpc.generatetoaddress(10, bob_address.address))
_ = list(brpc.generatetoaddress(120, miner_address))

# Give LNDs time to catch up
time.sleep(5)

In [7]:
print(f"""alice balance={ali.lnd.WalletBalance(lnmsg.WalletBalanceRequest())}
bob balance={bob.lnd.WalletBalance(lnmsg.WalletBalanceRequest())}""")

alice balance=total_balance: 13281142900
confirmed_balance: 13281142900
account_balance {
  key: "default"
  value {
    confirmed_balance: 13281142900
  }
}

bob balance=total_balance: 13265242900
confirmed_balance: 13265242900
account_balance {
  key: "default"
  value {
    confirmed_balance: 13265242900
  }
}



## Create Double-Funded Channel

In [8]:
!pip install -e ./p2oc
!p2oc --help

Obtaining file:///notebooks/p2oc
  Cloning https://github.com/flywheelstudio/python-bitcoinlib (to revision 0.19.0.1-support) to /tmp/pip-install-cvp7q6wb/python-bitcoinlib_9c3f8b8e56994280afec882536df1efa
  Running command git clone -q https://github.com/flywheelstudio/python-bitcoinlib /tmp/pip-install-cvp7q6wb/python-bitcoinlib_9c3f8b8e56994280afec882536df1efa
  Running command git checkout -b 0.19.0.1-support --track origin/0.19.0.1-support
  Switched to a new branch '0.19.0.1-support'
  Branch '0.19.0.1-support' set up to track remote branch '0.19.0.1-support' from 'origin'.
  Attempting uninstall: p2oc
    Found existing installation: p2oc 0.1
    Uninstalling p2oc-0.1:
      Successfully uninstalled p2oc-0.1
  Running setup.py develop for p2oc
Usage: p2oc [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  createchannelfromoffer  The offer we'll use to create a channel from.
  createoffer


### Alice (Taker)

Creates public offer to request inbound liquidity in exchange for a fee.

```bash
# from host
docker exec -it jupyter bash

LND_ENDPOINT="ali-lnd:10009" \
LND_CERTPATH="/ali-lnd/tls.cert" \
LND_MRNPATH="/tmp/ali.macaroon" \
    p2oc createoffer --request_fund_amount=16000000 --premium_amount=100000
```

### Bob (Maker)

Accepts offer and sends funding transaction response.

```bash
# from host
docker exec -it jupyter bash

LND_ENDPOINT="bob-lnd:10009" \
LND_CERTPATH="/bob-lnd/tls.cert" \
LND_MRNPATH="/tmp/bob.macaroon" \
    p2oc createchannelfromoffer <offer>
```

## Confirm Channel Opening

In [ ]:
_ = list(brpc.generatetoaddress(6, miner_address))

In [ ]:
# should be no pending channels
assert len(ali.lnd.PendingChannels(lnmsg.PendingChannelsRequest()).pending_open_channels) == 0

ali.lnd.ListChannels(lnmsg.ListChannelsRequest())